In [1]:
import faiss
faiss_path = "D:/rag_chatbot_covid_19/vectorstore/last_covid19_faiss.index"
index = faiss.read_index(faiss_path)
print("✅ FAISS index berhasil dimuat.")

✅ FAISS index berhasil dimuat.


In [5]:
import requests

try:
    requests.get("http://localhost:11434", timeout=3)
    print("✅ Ollama server aktif.")
except requests.exceptions.RequestException:
    print("❌ Ollama belum jalan. Jalankan di terminal: ollama serve")
    exit()


✅ Ollama server aktif.


In [ ]:
import ollama
from sentence_transformers import SentenceTransformer
import numpy as np, faiss, json

# --- Load embedding model ---
embedder = SentenceTransformer("intfloat/multilingual-e5-base")

# --- Load FAISS index ---
index = faiss.read_index("D:/rag_chatbot_covid_19/vectorstore/last_covid19_faiss.index")
with open("D:/rag_chatbot_covid_19/vectorstore/last_covid19_faiss_texts.json", "r", encoding="utf-8") as f:
    texts = json.load(f)

# --- Prompt sistem chatbot utama ---
system_prompt = """
Kamu adalah chatbot akademik yang menjawab hanya tentang COVID-19 di Indonesia.
Jangan menjawab hal yang tidak relevan seperti judi, hiburan, atau politik internasional.
Fokus pada analisis data provinsi, statistik, dan pandemi COVID-19.
"""

# --- Step 1: Query Adjustment ---
def adjust_query(question):
    prompt_adjust = """
Ubah pertanyaan berikut menjadi versi yang lebih singkat dan cocok untuk pencarian data COVID-19 di Indonesia.
Jangan tambahkan hal yang tidak relevan.
"""
    response = ollama.chat(model="phi3:mini", messages=[
        {"role": "system", "content": prompt_adjust},
        {"role": "user", "content": question},
    ])
    return response["message"]["content"]

# --- Step 2: RAG Chat ---
def rag_chat(question):
    # Adjust pertanyaan dulu
    adjusted = adjust_query(question)
    print(f"\n🔧 Adjusted Query: {adjusted}\n")

    # Embedding pertanyaan yang sudah di-adjust
    q_emb = embedder.encode([adjusted], convert_to_numpy=True).astype("float32")

    # Cari dokumen yang paling relevan di FAISS
    D, I = index.search(q_emb, k=3)
    contexts = "\n".join([texts[i] for i in I[0]])

    # Generate jawaban dari hasil retrieval
    response = ollama.chat(model="gemma3:1b", messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Gunakan informasi berikut:\n{contexts}\n\nPertanyaan: {question}"}
    ])
    return response["message"]["content"]

user_query = "Provinsi mana yang memiliki kasus COVID-19 tertinggi di tahun 2021?"

# Jalankan RAG
answer = rag_chat(user_query)
print("\n💬 Jawaban Chatbot:\n", answer)



🔧 Adjusted Query: Keprovinsi dengan kasus terbanyak COVID-19 di Indonesia pada tahun 2021 adalah DKI Jakarta.


💬 Jawaban Chatbot:
 Berdasarkan data yang diberikan, **DKI Jakarta** memiliki kasus COVID-19 tertinggi di tahun 2021.

Berikut adalah perbandingan total kasus di setiap provinsi:

*   **DKI Jakarta:** 167841 kasus
*   **Jawa Timur:** 161518 kasus
*   **Jawa Barat:** 161518 kasus
*   **Bali:** 14484 kasus
*   **Sumatera Barat:** 14484 kasus
*   **Sulawesi Selatan:** 14484 kasus
*   **Luminaria:** 135762 kasus

Dengan demikian, DKI Jakarta memiliki total kasus COVID-19 terbanyak dari semua provinsi.
